# Solving Hangman

## What is Hangman?

Hangman is a paper and pencil guessing game for two or more players. One player thinks of a word, phrase or sentence and the other tries to guess it by suggesting letters or numbers, within a certain number of guesses.

## Project Goals

* Explore several player strategies
* Explore several adversary strategies
* Compare different players vs. different adversaries
* Attempt to reach an equillibrium for the player on one side, and adversary on the other side

## Word list

Both the player and the adversary use the same known word list. For this work, we use a list included with `NLTK`, converted to lowercase to improve consistency of player and adversary strategies.

In [1]:
from nltk.corpus import words

wl = [w.lower() for w in words.words()]

## Code structure

The code is subdivided as follows:

* `Hangman.ipynb` - this notebook, containing discussion and experiments.
* `game.py`, implementing the `Game` class, which stores game parameters and rules.
* `player.py`, implementing various player strategies.
* `adversary.py`, implementing various adversary strategies.
* `functions.py`, implementing universal functions used throughout this project.
* `test.py`, containing testing and plotting automation


In [2]:
import game
import player
import adversary
import test

## Player Strategies: 

### Strategy 0: 'Drunken' Random Player

This is the easiest strategy to implement. The 'drunken' player always selects a random letter in hope of guessing the word. He is oblivious of any patterns or even the word list. Unfortunately, his chances are very low at ≤ 5% guessed words.

In [30]:
test.test(player.play0, adversary.adv0, wl, n=100, verbose=False)

2

### Strategy 1: Using fixed letter frequency in all words

Following the idea described in Edgar Poe's _Gold Bug_, if we examine all words in the word list, we notice that some letters appear much more often than others. For instance, `a` is found in more words than, say, `q`. 

Using this idea, this strategy combs through the entire word list and builds a letter frequency table (histogram). The player then makes a _greedy_ guess of the most frequent letters in the order they appear in the table.

This strategy yields a higher but still low success rate at ≤ 30% correct guesses.

In [31]:
test.test(player.play1, adversary.adv0, wl, n=100, verbose=False)

27

### Strategy 2: Using frequencies for words of known length

We can also notice that letter frequency is different in words of different lengths. In theory, this would allow the frequency-based player to guess more words. In practice, this refinement offers little advantage over the previous method.

In [32]:
test.test(player.play2, adversary.adv0, wl, n=100, verbose=False)

28

### Strategy 3: Word list bisect-based player using only correct letters

Understanding that there are inherent limitations to using letter frequency alone, we further attempt to devise a strategy that uses tbe correctly guessed letters and their known order (mask) to filter the word list for words matching this mask. One might argue that this strategy employs a high-level _binary search_ idea, because each iteration slices the remainig word list into valid and invalid parts.

In practice, this method provides a massive advantage against a naïve adversary, with a ca. 90% success rate.

In [33]:
test.test(player.play3, adversary.adv0, wl, n=100, verbose=False)

91

### Strategy 4: Word list bisect-based player using correct & incorrect letters

This strategy is very similar to strategy number 3, but it also uses incorrectly guessed letters to further narrow the list of candidate words.

The result seems to be slightly better than that with the previous strategy.

In [34]:
test.test(player.play4, adversary.adv0, wl, n=100, verbose=False)

96

## A Tougher Enemy

All of the above strategies were tested against a naïve adversary that randomly selects words form the word list. But what if we could suggest words that would give players a harder time to guess?

### Adversary 1: Known results of a previous play-through

Playing a certain strategy against the entire word list provides a 'score card' for this type of player. This strategy uses this score card to create a _discrete distribution_ biased towards more 'difficult' words. Of course, the success of this approach depends largely on the player strategy used to make the score card used as input.

For instance, observe the scores of previously mentioned player models against an adversary using Player 3's score card:

In [40]:
print(test.test(player.play0, adversary.adv1, wl, n=100, verbose=False))
print(test.test(player.play1, adversary.adv1, wl, n=100, verbose=False))
print(test.test(player.play2, adversary.adv1, wl, n=100, verbose=False))
print(test.test(player.play3, adversary.adv1, wl, n=100, verbose=False))
print(test.test(player.play4, adversary.adv1, wl, n=100, verbose=False))

3
13
18
76
85


As you can see, suggesting words that are more difficult predictably decreases player scores.

### Adversary 2: Assuming Letter Difficulty

This strategy tries to calculate difficulty of individual letters based on a known scorecard, and then maximize compound difficulty of letters in a defined selection. It doesn't use the scorecard _per se_ to select words, but rather, a calculated letter difficulty map. 

The resulting strategy seems to be stronger against letter frequency-based players, but to have almost no effect against bisect players.

In [5]:
print(test.test(player.play0, adversary.adv2, wl, n=100, verbose=False))
print(test.test(player.play1, adversary.adv2, wl, n=100, verbose=False))
print(test.test(player.play2, adversary.adv2, wl, n=100, verbose=False))
print(test.test(player.play3, adversary.adv2, wl, n=100, verbose=False))
print(test.test(player.play4, adversary.adv2, wl, n=100, verbose=False))

0
8
7
80
85


## Possible Future Improvements

* Selecting next letters based on _entropy_, i.e. how much information about the word the player can elicit through a guess;
* Attempting to maximize win probability using dynamic programming;
* Machine learning-based player and adversary.